<a href="https://colab.research.google.com/github/Pavel184/NLP_basic_course/blob/Homework_lesson_8/Homework_lesson_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HOMEWORK_LESSON_8

In [1]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=e41ff588ff2cf459cb29568c490e88234e2d381ca81a8aed29dcd8d8f6728322
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 14.1 MB/s 


In [3]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from tensorflow.keras.utils import plot_model 
import tensorflow as tf
import plotly.graph_objects as go

In [4]:
from google.colab import drive

drive.mount('/content/drive')  

Mounted at /content/drive


In [5]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/data_lesson_8/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/data_lesson_8/test.csv')
df_val = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/data_lesson_8/val.csv')

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [8]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [9]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [10]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [11]:
# Таблица результатов
models_results = {
    'approach': [],
    'Test_loss': [],
    'Test_accuracy': []
}

### 1. построить свёрточные архитектуры

In [12]:
epochs = 5

In [13]:
# Модель CNN
input = tf.keras.Input(shape=(training_length))
emb_1 = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb_1)
glob_max_pool = GlobalMaxPool1D()(conv_1)
dens_1 = Dense(10, activation='relu')(glob_max_pool)

output = Dense(1,activation='sigmoid' )(dens_1)

model_cnn = Model(inputs=input, outputs=output)

model_cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding (Embedding)       (None, 27, 32)            8259456   
                                                                 
 conv1d (Conv1D)             (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                             

In [14]:
history_cnn = model_cnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
319/319 [==============================] - 33s 102ms/step - loss: 0.5669 - accuracy: 0.6955 - val_loss: 0.4942 - val_accuracy: 0.7547
Epoch 2/5
319/319 [==============================] - 32s 101ms/step - loss: 0.3259 - accuracy: 0.8623 - val_loss: 0.5255 - val_accuracy: 0.7492
Epoch 3/5
319/319 [==============================] - 32s 101ms/step - loss: 0.1312 - accuracy: 0.9519 - val_loss: 0.6561 - val_accuracy: 0.7403
Epoch 4/5
319/319 [==============================] - 32s 101ms/step - loss: 0.0575 - accuracy: 0.9808 - val_loss: 0.7928 - val_accuracy: 0.7343
Epoch 5/5
319/319 [==============================] - 32s 101ms/step - loss: 0.0302 - accuracy: 0.9907 - val_loss: 0.9082 - val_accuracy: 0.7299


In [15]:
score_cnn = model_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn[0])
print('Test accuracy:', score_cnn[1])

45/45 [==============================] - 0s 5ms/step - loss: 1.0402 - accuracy: 0.7169


Test score: 1.0401943922042847
Test accuracy: 0.7168805003166199


In [16]:
models_results['approach'].append('CNN')
models_results['Test_loss'].append(score_cnn[0])
models_results['Test_accuracy'].append(score_cnn[1])

### 2. построить различные архитектуры с RNN

### SimpleRNN

In [17]:
model_rnn = Sequential()

model_rnn.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=32,
              trainable=True,
              mask_zero=True))
model_rnn.add(Masking(mask_value=0.0))

model_rnn.add(SimpleRNN(32, recurrent_dropout=0.2))
model_rnn.add(Dense(10, activation='relu'))
model_rnn.add(Dropout(0.25))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 masking (Masking)           (None, 27, 32)            0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 8,261,877
Trainable params: 8,261,877
Non-

In [18]:
history_rnn = model_rnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
319/319 [==============================] - 42s 128ms/step - loss: 0.6004 - accuracy: 0.6595 - val_loss: 0.5071 - val_accuracy: 0.7463
Epoch 2/5
319/319 [==============================] - 40s 126ms/step - loss: 0.3870 - accuracy: 0.8367 - val_loss: 0.5245 - val_accuracy: 0.7495
Epoch 3/5
319/319 [==============================] - 41s 127ms/step - loss: 0.2032 - accuracy: 0.9269 - val_loss: 0.6556 - val_accuracy: 0.7362
Epoch 4/5
319/319 [==============================] - 40s 126ms/step - loss: 0.1182 - accuracy: 0.9606 - val_loss: 0.7428 - val_accuracy: 0.7338
Epoch 5/5
319/319 [==============================] - 41s 128ms/step - loss: 0.0814 - accuracy: 0.9737 - val_loss: 0.9080 - val_accuracy: 0.7290


In [19]:
score_rnn = model_rnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_rnn[0])
print('Test accuracy:', score_rnn[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.9296 - accuracy: 0.7227


Test score: 0.9295567274093628
Test accuracy: 0.722655713558197


In [20]:
models_results['approach'].append('SimpleRNN')
models_results['Test_loss'].append(score_rnn[0])
models_results['Test_accuracy'].append(score_rnn[1])

### LSTM

In [21]:
model_lstm = Sequential()

model_lstm.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=32,
              trainable=True,
              mask_zero=True))
model_lstm.add(Masking(mask_value=0.0))
model_lstm.add(LSTM(32, recurrent_dropout=0.2))
model_lstm.add(Dense(10, activation='relu'))
model_lstm.add(Dropout(0.25))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_lstm.summary()

history_lstm = model_lstm.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 masking_1 (Masking)         (None, 27, 32)            0         
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense_4 (Dense)             (None, 10)                330       
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 8,268,117
Trainable params: 8,268,117
No

In [22]:
score_lstm = model_lstm.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_lstm[0])
print('Test accuracy:', score_lstm[1])

45/45 [==============================] - 1s 21ms/step - loss: 1.0397 - accuracy: 0.7183


Test score: 1.0396699905395508
Test accuracy: 0.7182912230491638


In [23]:
models_results['approach'].append('LSTM')
models_results['Test_loss'].append(score_lstm[0])
models_results['Test_accuracy'].append(score_lstm[1])

### GRU


In [24]:
model_gru = Sequential()

model_gru.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=32,
              trainable=True,
              mask_zero=True))
model_gru.add(Masking(mask_value=0.0))
model_gru.add(GRU(32, recurrent_dropout=0.2))
model_gru.add(Dense(10, activation='relu'))
model_gru.add(Dropout(0.25))
model_gru.add(Dense(1, activation='sigmoid'))

model_gru.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_gru.summary()

history_gru = model_gru.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 masking_2 (Masking)         (None, 27, 32)            0         
                                                                 
 gru (GRU)                   (None, 32)                6336      
                                                                 
 dense_6 (Dense)             (None, 10)                330       
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 8,266,133
Trainable params: 8,266,133
No

In [25]:
score_gru = model_gru.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_gru[0])
print('Test accuracy:', score_gru[1])

45/45 [==============================] - 1s 19ms/step - loss: 1.0394 - accuracy: 0.7142


Test score: 1.039439082145691
Test accuracy: 0.7141912579536438


In [26]:
models_results['approach'].append('GRU')
models_results['Test_loss'].append(score_gru[0])
models_results['Test_accuracy'].append(score_gru[1])

### 3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)

### SimpleRNN->CNN

In [27]:
input = tf.keras.Input(shape=(training_length))
emb_1 = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)

rnn = SimpleRNN(32, recurrent_dropout=0.2, return_sequences=True)(emb_1)

conv_1 = Conv1D(32, 3, activation='relu')(rnn)
glob_max_pool = GlobalMaxPool1D()(conv_1)



dens = Dense(10, activation='relu')(glob_max_pool)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_rnn_to_cnn = Model(inputs=input, outputs=output)

model_rnn_to_cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_rnn_to_cnn.summary()

history_rnn_to_cnn = model_rnn_to_cnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 27, 32)            2080      
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                           

In [28]:
score_rnn_to_cnn = model_rnn_to_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_rnn_to_cnn[0])
print('Test accuracy:', score_rnn_to_cnn[1])

45/45 [==============================] - 1s 12ms/step - loss: 0.9280 - accuracy: 0.7196


Test score: 0.9279695153236389
Test accuracy: 0.719569742679596


In [29]:
models_results['approach'].append('RNN->CNN')
models_results['Test_loss'].append(score_rnn_to_cnn[0])
models_results['Test_accuracy'].append(score_rnn_to_cnn[1])

### CNN->SimpleRNN

In [30]:
input = tf.keras.Input(shape=(training_length))
emb_1 = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb_1)
glob_max_pool = GlobalMaxPool1D(keepdims=True)(conv_1)

rnn = SimpleRNN(32, recurrent_dropout=0.2)(glob_max_pool)

dens = Dense(10, activation='relu')(rnn)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_cnn_to_rnn = Model(inputs=input, outputs=output)

model_cnn_to_rnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn_to_rnn.summary()

history_cnn_to_rnn = model_cnn_to_rnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding_5 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_2 (Glo  (None, 1, 32)            0         
 balMaxPooling1D)                                                
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 10)                330       
                                                           

In [31]:
score_cnn_to_rnn = model_cnn_to_rnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_to_rnn[0])
print('Test accuracy:', score_cnn_to_rnn[1])

45/45 [==============================] - 0s 6ms/step - loss: 1.2424 - accuracy: 0.7180


Test score: 1.2423659563064575
Test accuracy: 0.7179826498031616


In [32]:
models_results['approach'].append('CNN->RNN')
models_results['Test_loss'].append(score_cnn_to_rnn[0])
models_results['Test_accuracy'].append(score_cnn_to_rnn[1])

### CNN+SimpleRNN

In [33]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb)
glob_max_pool = GlobalMaxPool1D()(conv_1)

rnn = SimpleRNN(32, recurrent_dropout=0.2)(emb)

concat = Concatenate(   )([glob_max_pool, rnn])
dens = Dense(10, activation='relu')(concat)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_cnn_rnn = Model(inputs=input, outputs=output)

model_cnn_rnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn_rnn.summary()

history_cnn_rnn = model_cnn_rnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 27)]         0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 27, 32)       8259456     ['input_4[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 25, 32)       3104        ['embedding_6[0][0]']            
                                                                                                  
 global_max_pooling1d_3 (Global  (None, 32)          0           ['conv1d_3[0][0]']               
 MaxPooling1D)                                                                              

In [34]:
score_cnn_rnn = model_cnn_rnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_rnn[0])
print('Test accuracy:', score_cnn_rnn[1])

45/45 [==============================] - 0s 10ms/step - loss: 1.0737 - accuracy: 0.7211


Test score: 1.0737221240997314
Test accuracy: 0.7211127281188965


In [35]:
models_results['approach'].append('CNN+RNN')
models_results['Test_loss'].append(score_cnn_rnn[0])
models_results['Test_accuracy'].append(score_cnn_rnn[1])

### LSTM->CNN

In [36]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)

lstm = LSTM(32, recurrent_dropout=0.2, return_sequences=True)(emb)

conv_1 = Conv1D(32, 3, activation='relu')(lstm)
glob_max_pool = GlobalMaxPool1D()(conv_1)

dens = Dense(10, activation='relu')(glob_max_pool)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_lstm_to_cnn = Model(inputs=input, outputs=output)

model_lstm_to_cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_lstm_to_cnn.summary()

history_lstm_to_cnn = model_lstm_to_cnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding_7 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 lstm_1 (LSTM)               (None, 27, 32)            8320      
                                                                 
 conv1d_4 (Conv1D)           (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_4 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_14 (Dense)            (None, 10)                330       
                                                           

In [37]:
score_lstm_to_cnn = model_lstm_to_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_lstm_to_cnn[0])
print('Test accuracy:', score_lstm_to_cnn[1])

45/45 [==============================] - 1s 19ms/step - loss: 0.9913 - accuracy: 0.7169


Test score: 0.991269588470459
Test accuracy: 0.7168805003166199


In [38]:
models_results['approach'].append('LSTM->CNN')
models_results['Test_loss'].append(score_lstm_to_cnn[0])
models_results['Test_accuracy'].append(score_lstm_to_cnn[1])

### CNN->LSTM

In [39]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb)
glob_max_pool = GlobalMaxPool1D(keepdims=True)(conv_1)

lstm = LSTM(32, recurrent_dropout=0.2)(glob_max_pool)

dens = Dense(10, activation='relu')(lstm)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_cnn_to_lstm = Model(inputs=input, outputs=output)

model_cnn_to_lstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn_to_lstm.summary()

history_cnn_to_lstm = model_cnn_to_lstm.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding_8 (Embedding)     (None, 27, 32)            8259456   
                                                                 
 conv1d_5 (Conv1D)           (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_5 (Glo  (None, 1, 32)            0         
 balMaxPooling1D)                                                
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_16 (Dense)            (None, 10)                330       
                                                           

In [40]:
score_cnn_to_lstm = model_cnn_to_lstm.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_to_lstm[0])
print('Test accuracy:', score_cnn_to_lstm[1])

45/45 [==============================] - 0s 6ms/step - loss: 1.2951 - accuracy: 0.7100


Test score: 1.2951384782791138
Test accuracy: 0.7099589705467224


In [41]:
models_results['approach'].append('CNN->LSTM')
models_results['Test_loss'].append(score_cnn_to_lstm[0])
models_results['Test_accuracy'].append(score_cnn_to_lstm[1])

### CNN+LSTM

In [42]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb)
glob_max_pool = GlobalMaxPool1D()(conv_1)

lstm = LSTM(32, recurrent_dropout=0.2)(emb)

concat = Concatenate(   )([glob_max_pool, lstm])
dens = Dense(10, activation='relu')(concat)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_cnn_lstm = Model(inputs=input, outputs=output)

model_cnn_lstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn_lstm.summary()

history_cnn_lstm = model_cnn_lstm.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 27)]         0           []                               
                                                                                                  
 embedding_9 (Embedding)        (None, 27, 32)       8259456     ['input_7[0][0]']                
                                                                                                  
 conv1d_6 (Conv1D)              (None, 25, 32)       3104        ['embedding_9[0][0]']            
                                                                                                  
 global_max_pooling1d_6 (Global  (None, 32)          0           ['conv1d_6[0][0]']               
 MaxPooling1D)                                                                              

In [43]:
score_cnn_lstm = model_cnn_lstm.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_lstm[0])
print('Test accuracy:', score_cnn_lstm[1])

45/45 [==============================] - 1s 18ms/step - loss: 1.0211 - accuracy: 0.7205


Test score: 1.021109938621521
Test accuracy: 0.7204955220222473


In [44]:
models_results['approach'].append('CNN+LSTM')
models_results['Test_loss'].append(score_cnn_lstm[0])
models_results['Test_accuracy'].append(score_cnn_lstm[1])

### GRU->CNN

In [45]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)

gru = GRU(32, recurrent_dropout=0.2, return_sequences=True)(emb)

conv_1 = Conv1D(32, 3, activation='relu')(gru)
glob_max_pool = GlobalMaxPool1D()(conv_1)

dens = Dense(10, activation='relu')(glob_max_pool)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_gru_to_cnn = Model(inputs=input, outputs=output)

model_gru_to_cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_gru_to_cnn.summary()

history_gru_to_cnn = model_gru_to_cnn.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding_10 (Embedding)    (None, 27, 32)            8259456   
                                                                 
 gru_1 (GRU)                 (None, 27, 32)            6336      
                                                                 
 conv1d_7 (Conv1D)           (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_7 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_20 (Dense)            (None, 10)                330       
                                                           

In [46]:
score_gru_to_cnn = model_gru_to_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_gru_to_cnn[0])
print('Test accuracy:', score_gru_to_cnn[1])

45/45 [==============================] - 1s 18ms/step - loss: 0.8918 - accuracy: 0.7198


Test score: 0.8918284773826599
Test accuracy: 0.7198342084884644


In [47]:
models_results['approach'].append('GRU->CNN')
models_results['Test_loss'].append(score_gru_to_cnn[0])
models_results['Test_accuracy'].append(score_gru_to_cnn[1])

### CNN->GRU

In [48]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb)
glob_max_pool = GlobalMaxPool1D(keepdims=True)(conv_1)

gru = GRU(32, recurrent_dropout=0.2)(glob_max_pool)

dens = Dense(10, activation='relu')(gru)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_cnn_to_gru = Model(inputs=input, outputs=output)

model_cnn_to_gru.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn_to_gru.summary()

history_cnn_to_gru = model_cnn_to_gru.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 27)]              0         
                                                                 
 embedding_11 (Embedding)    (None, 27, 32)            8259456   
                                                                 
 conv1d_8 (Conv1D)           (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_8 (Glo  (None, 1, 32)            0         
 balMaxPooling1D)                                                
                                                                 
 gru_2 (GRU)                 (None, 32)                6336      
                                                                 
 dense_22 (Dense)            (None, 10)                330       
                                                           

In [49]:
score_cnn_to_gru = model_cnn_to_gru.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_to_gru[0])
print('Test accuracy:', score_cnn_to_gru[1])

45/45 [==============================] - 0s 6ms/step - loss: 1.2421 - accuracy: 0.7096


Test score: 1.2421464920043945
Test accuracy: 0.7096062898635864


In [50]:
models_results['approach'].append('CNN->GRU')
models_results['Test_loss'].append(score_cnn_to_gru[0])
models_results['Test_accuracy'].append(score_cnn_to_gru[1])

### CNN+GRU

In [51]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb)
glob_max_pool = GlobalMaxPool1D()(conv_1)

gru = GRU(32, recurrent_dropout=0.2)(emb)

concat = Concatenate(   )([glob_max_pool, gru])
dens = Dense(10, activation='relu')(concat)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model_cnn_gru = Model(inputs=input, outputs=output)

model_cnn_gru.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_cnn_gru.summary()

history_cnn_gru = model_cnn_gru.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 27)]         0           []                               
                                                                                                  
 embedding_12 (Embedding)       (None, 27, 32)       8259456     ['input_10[0][0]']               
                                                                                                  
 conv1d_9 (Conv1D)              (None, 25, 32)       3104        ['embedding_12[0][0]']           
                                                                                                  
 global_max_pooling1d_9 (Global  (None, 32)          0           ['conv1d_9[0][0]']               
 MaxPooling1D)                                                                              

In [52]:
score_cnn_gru = model_cnn_gru.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_gru[0])
print('Test accuracy:', score_cnn_gru[1])

45/45 [==============================] - 1s 18ms/step - loss: 1.0693 - accuracy: 0.7189


Test score: 1.0692641735076904
Test accuracy: 0.718908429145813


In [53]:
models_results['approach'].append('CNN+GRU')
models_results['Test_loss'].append(score_cnn_gru[0])
models_results['Test_accuracy'].append(score_cnn_gru[1])

### CNN->GRU->LSTM->SimpleRNN

In [54]:
input = tf.keras.Input(shape=(training_length))
emb = Embedding(input_dim=word_count, output_dim=32, input_length=training_length)(input)
conv_1 = Conv1D(32, 3, activation='relu')(emb)
glob_max_pool = GlobalMaxPool1D(keepdims=True)(conv_1)

gru = GRU(32, return_sequences=True, recurrent_dropout=0.2)(glob_max_pool)

lstm = LSTM(32, return_sequences=True, recurrent_dropout=0.2)(gru)

rnn = SimpleRNN(32, recurrent_dropout=0.2)(lstm)

dens = Dense(10, activation='relu')(rnn)
dens = Dropout(0.25)(dens)
output = Dense(1,activation='sigmoid' )(dens)

model = Model(inputs=input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 27)]              0         
                                                                 
 embedding_13 (Embedding)    (None, 27, 32)            8259456   
                                                                 
 conv1d_10 (Conv1D)          (None, 25, 32)            3104      
                                                                 
 global_max_pooling1d_10 (Gl  (None, 1, 32)            0         
 obalMaxPooling1D)                                               
                                                                 
 gru_4 (GRU)                 (None, 1, 32)             6336      
                                                                 
 lstm_4 (LSTM)               (None, 1, 32)             8320      
                                                          

In [55]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 1.5295 - accuracy: 0.7156


Test score: 1.5294790267944336
Test accuracy: 0.7156460881233215


In [56]:
models_results['approach'].append('CNN->GRU->LSTM->SimpleRNN')
models_results['Test_loss'].append(score_cnn_gru[0])
models_results['Test_accuracy'].append(score_cnn_gru[1])

### 4. сделать выводы что получилось лучше

In [57]:
# Итоговая таблица
df = pd.DataFrame(data=models_results).sort_values('Test_accuracy', ascending=False)
df.head(30)

,approach,Test_loss,Test_accuracy
1,SimpleRNN,0.929557,0.722656
6,CNN+RNN,1.073722,0.721113
9,CNN+LSTM,1.021110,0.720496
10,GRU->CNN,0.891828,0.719834
4,RNN->CNN,0.927970,0.719570
12,CNN+GRU,1.069264,0.718908
13,CNN->GRU->LSTM->SimpleRNN,1.069264,0.718908
2,LSTM,1.039670,0.718291
5,CNN->RNN,1.242366,0.717983
0,CNN,1.040194,0.716881


По итоговой таблице после 5 эпох обучения лучше показала SimpleRNN, но тут идёт явно переобучение.

In [74]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history_cnn.history['accuracy'],
                    name='cnn'))
fig.add_trace(go.Scattergl(y=history_rnn.history['accuracy'],
                    name='rnn'))
fig.add_trace(go.Scattergl(y=history_lstm.history['accuracy'],
                    name='lstm'))
fig.add_trace(go.Scattergl(y=history_gru.history['accuracy'],
                    name='gru'))
fig.add_trace(go.Scattergl(y=history_cnn_gru.history['accuracy'],
                    name='cnn+gru'))
fig.add_trace(go.Scattergl(y=history_cnn_lstm.history['accuracy'],
                    name='cnn+lstm'))
fig.add_trace(go.Scattergl(y=history_cnn_rnn.history['accuracy'],
                    name='cnn+rnn'))
fig.add_trace(go.Scattergl(y=history_cnn_to_gru.history['accuracy'],
                    name='cnn->gru'))
fig.add_trace(go.Scattergl(y=history_cnn_to_lstm.history['accuracy'],
                    name='cnn->lstm'))
fig.add_trace(go.Scattergl(y=history_cnn_to_rnn.history['accuracy'],
                    name='cnn->rnn'))
fig.add_trace(go.Scattergl(y=history_gru_to_cnn.history['accuracy'],
                    name='gru->cnn', line_color='black'))
fig.add_trace(go.Scattergl(y=history_lstm_to_cnn.history['accuracy'],
                    name='lstm->cnn', line_color='brown'))
fig.add_trace(go.Scattergl(y=history_rnn_to_cnn.history['accuracy'],
                    name='rnn->cnn', line_color='gray'))
fig.add_trace(go.Scattergl(y=history.history['accuracy'],
                    name='CNN->GRU->LSTM->SimpleRNN', line_color='green'))


fig.update_layout(height=900, width=1100,
                  xaxis_title='Epoch',
                  yaxis_title='Train Accuracy')
fig.show()

Из графиков выше видно, что быстрее всех переобучаются CNN->GRU->LSTM->SimpleRNN, CNN->RNN

In [73]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history_cnn.history['val_accuracy'],
                    name='cnn'))
fig.add_trace(go.Scattergl(y=history_rnn.history['val_accuracy'],
                    name='rnn'))
fig.add_trace(go.Scattergl(y=history_lstm.history['val_accuracy'],
                    name='lstm'))
fig.add_trace(go.Scattergl(y=history_gru.history['val_accuracy'],
                    name='gru'))
fig.add_trace(go.Scattergl(y=history_cnn_gru.history['val_accuracy'],
                    name='cnn+gru'))
fig.add_trace(go.Scattergl(y=history_cnn_lstm.history['val_accuracy'],
                    name='cnn+lstm'))
fig.add_trace(go.Scattergl(y=history_cnn_rnn.history['val_accuracy'],
                    name='cnn+rnn'))
fig.add_trace(go.Scattergl(y=history_cnn_to_gru.history['val_accuracy'],
                    name='cnn->gru'))
fig.add_trace(go.Scattergl(y=history_cnn_to_lstm.history['val_accuracy'],
                    name='cnn->lstm'))
fig.add_trace(go.Scattergl(y=history_cnn_to_rnn.history['val_accuracy'],
                    name='cnn->rnn'))
fig.add_trace(go.Scattergl(y=history_gru_to_cnn.history['val_accuracy'],
                    name='gru->cnn', line_color='black'))
fig.add_trace(go.Scattergl(y=history_lstm_to_cnn.history['val_accuracy'],
                    name='lstm->cnn', line_color='brown'))
fig.add_trace(go.Scattergl(y=history_rnn_to_cnn.history['val_accuracy'],
                    name='rnn->cnn', line_color='gray'))
fig.add_trace(go.Scattergl(y=history.history['val_accuracy'],
                    name='CNN->GRU->LSTM->SimpleRNN', line_color='green'))


fig.update_layout(height=900, width=1100,
                  xaxis_title='Epoch',
                  yaxis_title='Val Accuracy')
fig.show()

Но если смотреть результат на валидационной подвыборке, то самая высокая точность на первой эпохе(до того как сеть начала переобучаться) видна у LSTM, CNN, CNN->GRU->LSTM->SimpleRNN Схожую точность на второй эпохе показала CNN+RNN.

Вывод: В зависимости от данных задачи, нужно пробовать различные конфигурации CNN, RNN, CNN->RNN, RNN->CNN и смотреть что работает лучше. В данном примере рекурсивные сети сами по себе, либо в сочетании со свёрточными показали себя несколько лучше, чем просто свёрточная сеть.